In [ ]:
# -*- coding: utf-8 -*-
"""Runner_py.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1LRg0rAa7OQds0BG9eBRE3wBkqoG6f7jR
"""

#Pipeline For Match
import pandas as pd
import warnings
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
import numpy as np
import requests
from bs4 import BeautifulSoup
import math
import requests
import pandas as pd
from bs4 import BeautifulSoup
import random
import numpy as np
import os
import sys
import openpyxl
from collections import defaultdict

warnings.filterwarnings("ignore")

class ExtendableLabelEncoder:
    def __init__(self):
        self.label_encoder = LabelEncoder()
        self.classes_ = []
        self.class_to_index = {}

    def fit(self, data):
        self.label_encoder.fit(data)
        self.classes_ = list(self.label_encoder.classes_)
        self.class_to_index = {label: i for i, label in enumerate(self.classes_)}
        return self

    def transform(self, data):
        result = []
        for val in data:
            if val in self.class_to_index:
                result.append(self.class_to_index[val])
            else:
                # Add new label
                new_index = len(self.classes_)
                self.class_to_index[val] = new_index
                self.classes_.append(val)
                result.append(new_index)
        return result

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

    def inverse_transform(self, indices):
        return [self.classes_[i] for i in indices]

    def get_classes(self):
        return self.classes_

#Train
def extract_player_data(player_id):
    player_id = str(player_id)
    #batting Dataframe
    batting_url = "https://stats.espncricinfo.com/ci/engine/player/"+player_id+".html?class=6;orderby=start;orderbyad=reverse;template=results;type=batting;view=innings"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response_batting = requests.get(batting_url, headers=headers)

    if response_batting.status_code == 200:
        soup = BeautifulSoup(response_batting.text, 'html.parser')
        rows = soup.find_all("tr", class_="data1")

        data = []
        for row in rows:
                cols = row.find_all("td")
                runs = cols[0].text.strip()
                balls = cols[2].text.strip()
                strike_rate = cols[5].text.strip()
                fours = cols[3].text.strip()
                sixes = cols[4].text.strip()
                opposition = cols[10].text.strip().replace("v ", "")
                ground = cols[11].text.strip()
                date = cols[12].text.strip()
                inning = cols[8].text.strip()
                data.append([runs, balls, strike_rate, fours, sixes, opposition, ground, date,inning])

        data = data[1:]
        for i in range(len(data)):
            for j in range(len(data[i])):
                if data[i][j] == '-':
                    data[i][j] = 0

        Batting = pd.DataFrame(data, columns=["Runs scored", "Balls", "Strike Rate", "4s", "6s", "Opposition", "Ground", "Date","inning"])
    else:
        #print("Failed to retrieve the webpage")
        c=54

    #bowling Dataframe
    bowling_url = "https://stats.espncricinfo.com/ci/engine/player/"+player_id+".html?class=6;orderby=start;orderbyad=reverse;template=results;type=bowling;view=match"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response_bowling = requests.get(bowling_url, headers=headers)

    if response_bowling.status_code == 200:
        soup = BeautifulSoup(response_bowling.text, 'html.parser')
        rows = soup.find_all("tr", class_="data1")
        th_tags = soup.find_all('th')
        column_names = [th.get_text(strip=True) for th in th_tags]
        b = 1 if any(col in ["Balls per over", "BPO"] for col in column_names) else 0
        data = []
        if b==1:
          for row in rows:
              cols = row.find_all("td")
              if len(cols) >= 10:
                  overs = cols[0].text.strip()
                  mdns = cols[2].text.strip()
                  runs = cols[3].text.strip()
                  wkts = cols[4].text.strip()
                  econ = cols[5].text.strip()
                  ave = cols[6].text.strip()
                  sr = cols[7].text.strip()
                  opposition = cols[8].text.strip().replace("v ", "")
                  ground = cols[9].text.strip()
                  date = cols[10].text.strip()

                  data.append([overs, mdns, runs, wkts, econ, ave, sr, opposition, ground, date])
        else:
           for row in rows:
              cols = row.find_all("td")
              if len(cols) >= 10:
                  overs = cols[0].text.strip()
                  mdns = cols[1].text.strip()
                  runs = cols[2].text.strip()
                  wkts = cols[3].text.strip()
                  econ = cols[4].text.strip()
                  ave = cols[5].text.strip()
                  sr = cols[6].text.strip()
                  opposition = cols[7].text.strip().replace("v ", "")
                  ground = cols[8].text.strip()
                  date = cols[9].text.strip()

                  data.append([overs, mdns, runs, wkts, econ, ave, sr, opposition, ground, date])

        data = data[1:]
        for i in range(len(data)):
            for j in range(len(data[i])):
                if data[i][j] == '-':
                    data[i][j] = 0

        Bowling = pd.DataFrame(data, columns=["Overs", "Mdns", "Runs concede", "Wkts", "Econ", "Ave", "SR", "Opposition", "Ground", "Date"])
        # Bowling.drop(columns=['Opposition'], inplace=True)
        # Bowling['Oppositon'] = Bowling['Ground']
        # Bowling['Ground'] = Bowling['Date']
        # Bowling.drop(columns=['Date'], inplace=True)
        # Bowling['Date'] = pd.to_datetime(Bowling['Date'], format='%d %b %Y')
        #display(Bowling)
    else:
        #print("Failed to retrieve the webpage")
        c=56
    #Fielding
    fielding_url = "https://stats.espncricinfo.com/ci/engine/player/"+player_id+".html?class=6;orderby=start;orderbyad=reverse;template=results;type=fielding;view=match"

    # Headers to mimic a real browser request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response_fielding = requests.get(fielding_url, headers=headers)

    if response_fielding.status_code == 200:
        soup = BeautifulSoup(response_fielding.text, 'html.parser')
        rows = soup.find_all("tr", class_="data1")

        data = []
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 9:
                dis = cols[0].text.strip()
                ct = cols[1].text.strip()
                st = cols[2].text.strip()
                ct_wk = cols[3].text.strip()
                ct_fi = cols[4].text.strip()
                opposition = cols[6].text.strip().replace("v ", "")
                ground = cols[7].text.strip()
                date = cols[8].text.strip()
                data.append([dis, ct, st, ct_wk, ct_fi, opposition, ground, date])

        data = data[1:]

        for i in range(len(data)):
            for j in range(len(data[i])):
                if data[i][j] == '-':
                    data[i][j] = 0

        Fielding = pd.DataFrame(data, columns=["dis", "Catches", "Stumpings", "Catches as Wk", "Catches as Fielder", "Opposition", "Ground", "Date"])
        Fielding["Run Outs"] = Fielding["dis"].astype(int) - Fielding["Catches"].astype(int) - Fielding["Stumpings"].astype(int)
    else:
        # print("Failed to retrieve the webpage")
        c=52

    Bowling.drop(columns=['Opposition','Ground','Date'], inplace=True)
    Fielding.drop(columns=['Opposition','Ground','Date'], inplace=True)
    all_stats = pd.concat([Batting, Bowling, Fielding], axis=1)
    return all_stats

#Calculating Fantasy Points
#Batting Fantasy Points

batting_scoring_system = {
    "Run": 1,
    "Boundary Bonus": 4,
    "Six Bonus": 6,
    "25 Run Bonus": 4,
    "Half-Century Bonus": 8,
    "75 Run Bonus": 12,
    "Century Bonus": 16,
    "Dismissal for a duck (Batter, Wicket-Keeper, & All-Rounder)": -2,
    "Strike Rate Above 170": 6,
    "Strike Rate Between 150.01-170": 4,
    "Strike Rate Between 130-150": 2,
    "Strike Rate Between 60-70": -2,
    "Strike Rate Between 50-59.99": -4,
    "Strike Rate Below 50": -6
}

def Batting_fantasy_points(player_data):
    # Ensure 'Runs' is treated as string before replacing '*'
    # player_data['Runs'] = player_data['Runs'].str.replace('*', '', regex=False).astype(int)

    for i in range(len(player_data['Runs scored'])):
        not_out = 0
        dnb = 0
        if len(player_data['Runs scored'][i])>=2 and player_data['Runs scored'][i][-1] == '*':
            player_data['Runs scored'][i] = player_data['Runs scored'][i][:-1]
            not_out = 1
        if player_data['Runs scored'][i] == 'DNB' or player_data['Runs scored'][i] == 'sub' or player_data['Runs scored'][i] == 'TDNB':
           player_data.loc[i, 'Runs scored'] = '0'
           dnb=1
        player_data['Not Out'] = not_out
        player_data['dnb']=dnb
    player_data['Runs scored'] = player_data['Runs scored'] = pd.to_numeric(player_data['Runs scored'], errors='coerce').fillna(0).astype(int)
    # Convert other numerical columns
    player_data['4s'] = player_data['4s'].astype(int)
    player_data['6s'] = player_data['6s'].astype(int)
    player_data['Strike Rate'] = player_data['Strike Rate'].astype(float)

    # Base Fantasy Points Calculation
    player_data['Batting Fantasy Points'] = (
        player_data['Runs scored'] * batting_scoring_system["Run"] +
        player_data['4s'] * batting_scoring_system["Boundary Bonus"] +
        player_data['6s'] * batting_scoring_system["Six Bonus"]
    )

    # Bonus Points for Milestones
    player_data['Batting Fantasy Points'] += player_data['Runs scored'].apply(lambda x:
        batting_scoring_system["Century Bonus"] if x >= 100 else
        batting_scoring_system["75 Run Bonus"] if x >= 75 else
        batting_scoring_system["Half-Century Bonus"] if x >= 50 else
        batting_scoring_system["25 Run Bonus"] if x >= 25 else 0
    )

    # Duck Penalty
    player_data['Batting Fantasy Points'] += player_data.apply(lambda row:
        batting_scoring_system["Dismissal for a duck (Batter, Wicket-Keeper, & All-Rounder)"]
        if row['Runs scored'] == 0 and row['Not Out']==0 and row['dnb']==0  else 0, axis=1
    )

    # Strike Rate Bonus/Penalty
    player_data['Balls'] = player_data['Balls'].astype(int)
    player_data['Batting Fantasy Points'] += player_data.apply(lambda row:
        # Apply strike rate bonus only if Balls Faced > 10
        batting_scoring_system["Strike Rate Above 170"] if row['Balls'] > 10 and row['Strike Rate'] > 170 else
        batting_scoring_system["Strike Rate Between 150.01-170"] if row['Balls'] > 10 and row['Strike Rate'] > 150 else
        batting_scoring_system["Strike Rate Between 130-150"] if row['Balls'] > 10 and row['Strike Rate'] > 130 else
        batting_scoring_system["Strike Rate Between 60-70"] if row['Balls'] > 10 and 60 <= row['Strike Rate'] < 70 else
        batting_scoring_system["Strike Rate Between 50-59.99"] if row['Balls'] > 10 and 50 <= row['Strike Rate'] < 60 else
        batting_scoring_system["Strike Rate Below 50"] if row['Balls'] > 10 and row['Strike Rate'] < 50 else 0
    , axis=1)

    return player_data

bowling_scoring_system = {
    "Wicket": 25,
    "Maiden Over": 12,
    "3 Wicket Bonus": 4,
    "4 Wicket Bonus": 8,
    "5 Wicket Bonus": 12,
    "Dot Ball": 1,
    "Economy Below 5": 6,
    "Economy Between 5-6": 4,
    "Economy Between 6-7": 2,
    "Economy Between 10-11": -2,
    "Economy Between 11-12": -4,
    "Economy Above 12": -6
}

def Bowling_fantasy_points(player_data):
    """
    Calculate Bowling Fantasy Points for a DataFrame of players.
    """

    # Convert relevant columns to numeric, handling missing values
    player_data['Overs'] = pd.to_numeric(player_data['Overs'], errors="coerce").fillna(0)
    player_data['Mdns'] = pd.to_numeric(player_data['Mdns'], errors="coerce").fillna(0)
    player_data['Runs concede'] = pd.to_numeric(player_data['Runs concede'], errors="coerce").fillna(0)
    player_data['Wkts'] = pd.to_numeric(player_data['Wkts'], errors="coerce").fillna(0)
    player_data['Econ'] = pd.to_numeric(player_data['Econ'], errors="coerce")

    # Convert overs into total balls bowled
    total_balls = player_data['Overs'].apply(lambda x: int(x) * 6 + round((x - int(x)) * 10))

    # Dot Ball Calculation (estimated, since exact dot ball data is often unavailable)
    player_data['Dot Balls'] = player_data.apply(lambda row: max((9 / row['Econ']) * (row['Wkts'] + 4), 0)
                                                 if row['Econ'] > 0 else 0, axis=1)

    # Round up the Dot Balls values using math.ceil
    player_data['Dot Balls'] = player_data['Dot Balls'].apply(math.ceil)

    # Ensure Dot Balls are not greater than total balls / 2
    player_data['Dot Balls'] = player_data.apply(lambda row: min(row['Dot Balls'], total_balls[row.name] // 2), axis=1)

    # Base Fantasy Points Calculation
    player_data['Bowling Fantasy Points'] = (
        player_data['Dot Balls'] * bowling_scoring_system["Dot Ball"] +
        player_data['Wkts'] * bowling_scoring_system["Wicket"] +
        player_data['Mdns'] * bowling_scoring_system["Maiden Over"]
    )

    # Bonus Points for Wicket Milestones
    player_data['Bowling Fantasy Points'] += player_data['Wkts'].apply(lambda x:
        bowling_scoring_system["5 Wicket Bonus"] if x >= 5 else
        bowling_scoring_system["4 Wicket Bonus"] if x >= 4 else
        bowling_scoring_system["3 Wicket Bonus"] if x >= 3 else 0
    )

    # Economy Rate Bonus/Penalty (Only if bowled at least 2 overs)
    player_data['Bowling Fantasy Points'] += player_data.apply(lambda row:
        bowling_scoring_system["Economy Below 5"] if row['Overs'] >= 2 and row['Econ'] < 5 else
        bowling_scoring_system["Economy Between 5-6"] if row['Overs'] >= 2 and 5 <= row['Econ'] < 6 else
        bowling_scoring_system["Economy Between 6-7"] if row['Overs'] >= 2 and 6 <= row['Econ'] < 7 else
        bowling_scoring_system["Economy Between 10-11"] if row['Overs'] >= 2 and 10 <= row['Econ'] < 11 else
        bowling_scoring_system["Economy Between 11-12"] if row['Overs'] >= 2 and 11 <= row['Econ'] < 12 else
        bowling_scoring_system["Economy Above 12"] if row['Overs'] >= 2 and row['Econ'] >= 12 else 0, axis=1
    )

    return player_data

fielding_scoring_system = {
    "Catch": 8,
    "3 Catch Bonus": 4,
    "Stumping": 12,
    "Run out": 8,  # General run out (not differentiating direct or not)
}

def Fielding_fantasy_points(player_data):
    """
    Calculate Fielding Fantasy Points for a DataFrame of players.
    """

    # Convert relevant columns to numeric, handling missing values
    player_data['Run Outs'] = pd.to_numeric(player_data['Run Outs'], errors="coerce").fillna(0)
    player_data['Catches'] = pd.to_numeric(player_data['Catches'], errors="coerce").fillna(0)
    player_data['Stumpings'] = pd.to_numeric(player_data['Stumpings'], errors="coerce").fillna(0)
    player_data['Catches as Wk'] = pd.to_numeric(player_data['Catches as Wk'], errors="coerce").fillna(0)
    player_data['Catches as Fielder'] = pd.to_numeric(player_data['Catches as Fielder'], errors="coerce").fillna(0)

    # Base Fantasy Points Calculation
    player_data['Fielding Fantasy Points'] = (
        player_data['Stumpings'] * fielding_scoring_system["Stumping"] +
        player_data['Catches as Wk'] * fielding_scoring_system["Catch"] +
        player_data['Catches as Fielder'] * fielding_scoring_system["Catch"] +
        player_data['Run Outs'] * fielding_scoring_system["Run out"]  # 8 points for run outs
    )

    # Bonus for 3 catches (for simplicity, you can adjust based on your rules)
    player_data['Fielding Fantasy Points'] += player_data['Catches'].apply(lambda x:
        fielding_scoring_system["3 Catch Bonus"] if x >= 3 else 0
    )

    return player_data

def custom_weighted_form(values, decay=0.9):
    weights = [decay**i for i in range(len(values))]
    numerator = sum(v * w for v, w in zip(values, weights))
    denominator = sum(weights)
    return numerator / denominator
def calculate_inning_feature(player_id, innings_counts, batting_innings, match_data, date, ground=None, opposition=None):
    data_of_player = match_data
    data_of_player['Date'] = pd.to_datetime(data_of_player['Date'], errors='coerce', dayfirst=True)
    data_of_player = data_of_player.dropna(subset=['Date'])
    data_of_player = data_of_player[data_of_player['Date'] < pd.to_datetime(date)]

    feature_values = {}
    #feature_values['Actual Points'] = data_of_player['Fantasy Points'].iloc[0] if not match_data.empty else 0 #Handle empty match_data

    for count in innings_counts:
        feature_values[f'F_rm_{count}'] = data_of_player["Fantasy Points"].iloc[:count].mean() if not data_of_player.empty else 0 #Handle empty data_of_player
        feature_values[f'F_ewm_{count}'] = custom_weighted_form(data_of_player["Fantasy Points"].iloc[:count], decay=0.7) if not data_of_player.empty else 0 #Handle empty data_of_player

    data_of_player_innings = data_of_player[data_of_player['inning'].astype(int) == int(batting_innings)]
    data_of_player_innings = data_of_player_innings.sort_values(by="Date", ascending=False)

    for count in innings_counts:
        feature_values[f'I_rm_{count}'] = data_of_player_innings["Fantasy Points"].iloc[:count].mean() if not data_of_player_innings.empty else 0 #Handle empty data_of_player_innings
        feature_values[f'I_ewm_{count}'] = custom_weighted_form(data_of_player_innings["Fantasy Points"].iloc[:count], decay=0.9) if not data_of_player_innings.empty else 0 #Handle empty data_of_player_innings

    if ground is not None:
        data_of_player_ground = data_of_player[data_of_player['Ground'] == ground]

        for count in innings_counts:
            feature_values[f'G_rm_{count}'] = data_of_player_ground["Fantasy Points"].iloc[:count].mean() if not data_of_player_ground.empty else 0 #Handle empty data_of_player_ground
            # ewm_result = data_of_player_ground['Fantasy Points'].iloc[:count].ewm(span=count, adjust=False).mean()
            feature_values[f'G_ewm_{count}'] = custom_weighted_form(data_of_player_ground["Fantasy Points"].iloc[:count], decay=0.9) if not data_of_player_ground.empty else 0 #Handle empty ewm_result

    if opposition is not None:
        data_of_player_opposition = data_of_player[data_of_player['Opposition'] == opposition]
        #display(player_id,data_of_player_opposition)
        for count in innings_counts:
            feature_values[f'O_rm_{count}'] = data_of_player_opposition["Fantasy Points"].iloc[:count].mean() if not data_of_player_opposition.empty else 0  #Handle empty data_of_player_opposition
            #ewm_result = data_of_player_opposition['Fantasy Points'].iloc[:count].ewm(span=count, adjust=False).mean()
            feature_values[f'O_ewm_{count}'] = custom_weighted_form(data_of_player_opposition["Fantasy Points"].iloc[:count], decay=0.95) if not data_of_player_opposition.empty else 0  #Handle empty ewm_result

    return feature_values


innings_counts = [3, 5, 10, 15]


def training_pipeline(match_date):
  #Training
  from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
  regular_players = pd.read_csv('Regular.csv')
  regular_players_ids = regular_players['ID']
  all_data_list = []
  date_of_match = match_date
  date_of_match = pd.to_datetime(date_of_match)
  for i in range(len(regular_players_ids)):
    try:
      player_data = extract_player_data(regular_players_ids[i])
      player_data = Batting_fantasy_points(player_data)
      player_data = Bowling_fantasy_points(player_data)
      player_data = Fielding_fantasy_points(player_data)
      player_data['Date'] = pd.to_datetime(player_data['Date'], format="%d %b %Y")
      player_data = player_data[pd.to_datetime(player_data['Date']) < date_of_match]
      player_data["Fantasy Points"] = (
          player_data["Batting Fantasy Points"]
          + player_data["Bowling Fantasy Points"]
          + player_data["Fielding Fantasy Points"]
          + 4
      )

      for j in range(min(10, len(player_data))):
          feature_values = calculate_inning_feature(
              regular_players_ids[i],
              innings_counts,
              int(player_data['inning'].iloc[j]),
              player_data,
              pd.to_datetime(player_data['Date'].iloc[j]),
              player_data['Ground'].iloc[j],
              player_data['Opposition'].iloc[j]
          )

          # Add extra match details
          feature_values.update({
              'Player ID': regular_players_ids[i],
              'Date': player_data['Date'].iloc[j],
              'Ground': player_data['Ground'].iloc[j],
              'Opposition': player_data['Opposition'].iloc[j],
              'inning': player_data['inning'].iloc[j],
              'Fantasy Points': player_data['Fantasy Points'].iloc[j]
          })

          all_data_list.append(feature_values)
    except Exception as e:
      #print(f"Error occurred: {e}")
      c=54

  # #Dataset Ready to Train
  all_player_match_wise_data = pd.DataFrame(all_data_list)
  return all_player_match_wise_data

CSK = ["2025-03-23", "2025-03-28", "2025-03-30", "2025-04-05", "2025-04-08", "2025-04-11", "2025-04-14", "2025-04-20", "2025-04-25", "2025-04-30", "2025-05-03", "2025-05-07", "2025-05-12", "2025-05-18"]
DC = ["2025-03-24", "2025-03-30", "2025-04-05", "2025-04-10", "2025-04-13", "2025-04-16", "2025-04-19", "2025-04-22", "2025-04-27", "2025-04-29", "2025-05-05", "2025-05-08", "2025-05-11", "2025-05-15"]
MI = ["2025-03-23", "2025-03-29", "2025-03-31", "2025-04-04", "2025-04-07", "2025-04-13", "2025-04-17", "2025-04-20", "2025-04-23", "2025-04-27", "2025-05-01", "2025-05-06", "2025-05-11", "2025-05-15"]
PBKS = ["2025-03-25", "2025-04-01", "2025-04-05", "2025-04-08", "2025-04-12", "2025-04-15", "2025-04-18", "2025-04-20", "2025-04-26", "2025-04-30", "2025-05-04", "2025-05-08", "2025-05-11", "2025-05-16"]
GT = ["2025-03-25", "2025-03-29", "2025-04-02", "2025-04-06", "2025-04-09", "2025-04-12", "2025-04-19", "2025-04-21", "2025-04-28", "2025-05-02", "2025-05-06", "2025-05-11", "2025-05-14", "2025-05-18"]
RCB = ["2025-03-22", "2025-03-28", "2025-04-02", "2025-04-07", "2025-04-10", "2025-04-13", "2025-04-18", "2025-04-20", "2025-04-24", "2025-04-27", "2025-05-03", "2025-05-09", "2025-05-13", "2025-05-17"]
SRH = ["2025-03-23", "2025-03-27", "2025-03-30", "2025-04-03", "2025-04-06", "2025-04-12", "2025-04-17", "2025-04-23", "2025-04-25", "2025-05-02", "2025-05-05", "2025-05-10", "2025-05-13", "2025-05-18"]
LSG = ["2025-03-24", "2025-03-27", "2025-04-01", "2025-04-04", "2025-04-08", "2025-04-12", "2025-04-14", "2025-04-19", "2025-04-22", "2025-04-27", "2025-05-04", "2025-05-09", "2025-05-14", "2025-05-18"]
RR = ["2025-03-23", "2025-03-26", "2025-03-30", "2025-04-05", "2025-04-09", "2025-04-13", "2025-04-16", "2025-04-19", "2025-04-24", "2025-04-28", "2025-05-01", "2025-05-04", "2025-05-12", "2025-05-16"]
KKR = ["2025-03-22", "2025-03-26", "2025-03-31", "2025-04-03", "2025-04-08", "2025-04-11", "2025-04-15", "2025-04-21", "2025-04-26", "2025-04-29", "2025-05-04", "2025-05-07", "2025-05-10", "2025-05-17"]

team_dates = {"CSK": CSK, "DC": DC, "MI": MI, "PBKS": PBKS, "GT": GT,
              "RCB": RCB, "SRH": SRH, "LSG": LSG, "RR": RR, "KKR": KKR}
def impacts(sq_data,date,match_no):
    sq_data['Team'] = sq_data['Team'].replace('CHE', 'CSK')
    xfactors = sq_data[sq_data['IsPlaying'] == 'X_FACTOR_SUBSTITUTE']
    teams = xfactors['Team'].unique()
    details = print_match_details(match_no, teams)
    today_str = details['Date']
    # Map teams
    batting_first = team_mapping.get(details['1st'], 'Unknown')
    if batting_first == 'Unknown':
        batting_first = teams[0]
    batting_second = teams[1] if teams[0] == batting_first else teams[0]
    # print(f"Batting First: {batting_first}")
    # print(f"Batting Second: {batting_second}")

    # Find last match date before today for each team
    # last_played =
    last_played = {}
    for team in teams:
        team = 'CSK' if team == 'CHE' else team
        dates = pd.to_datetime(team_dates[team])
        past_dates = dates[dates < pd.to_datetime(today_str)]
        last_date = past_dates.max() if not past_dates.empty else "No previous match"
        last_played[team] = last_date.strftime('%Y-%m-%d') if last_date != "No previous match" else last_date

    final_news = pd.read_csv('final_new.csv')
    name_to_id = final_news.set_index('Player Name')['Player ID'].to_dict()
    xfactors['Player ID'] = xfactors['Player Name'].map(name_to_id).astype('Int64')

    expected = {}
    for _, row in xfactors.iterrows():
        team = row['Team']
        player_id = row['Player ID']
        try:
            player_stats = extract_player_data(player_id)
            player_stats = Batting_fantasy_points(player_stats)
            player_stats = Bowling_fantasy_points(player_stats)
            player_stats['Date'] = pd.to_datetime(player_stats['Date'], format="%d %b %Y")
            player_stats = player_stats.dropna(subset=['Date'])
            if last_played[team] in player_stats['Date'].tolist():
                expected[row['Player Name']] = team
        except Exception as e:
            print(f"Error processing {row['Player Name']} → {e}")
            c=3

    team_to_players = defaultdict(list)
    for player, team in expected.items():
        team_to_players[team].append(player)

    #print("\n--- Selected Players Based on Last 5 Innings Fantasy Points ---")
    final_names=[]
    for team, players in team_to_players.items():
        scores = []
        for player in players:
            player_id = name_to_id.get(player)
            if player_id is None:
                continue
            try:
                stats = extract_player_data(player_id)
                stats = Batting_fantasy_points(stats)
                stats = Bowling_fantasy_points(stats)
                stats['Date'] = pd.to_datetime(stats['Date'], format="%d %b %Y")
                stats = stats[stats['Date'] < pd.to_datetime(today_str)]
                stats = stats.dropna(subset=['Date']).sort_values('Date', ascending=False)
                stats = stats.head(5)

                if team == batting_first:
                    avg_score = stats['Bowling Fantasy Points'].mean()
                else:
                    avg_score = stats['Batting Fantasy Points'].mean()

                scores.append((player, avg_score))
            except Exception as e:
                # print(f"Error evaluating {player} → {e}")
                c=8

        scores = sorted(scores, key=lambda x: x[1], reverse=True)
        # print(f"{team}:")
        # print(scores[0][0])
        final_names.append(scores[0][0])
    return final_names

def pipeline_squad_data(match_date,csv_name,all_player_match_wise_data,batting_team,ground,match_no):
  #Training
  from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
  date_of_match = pd.to_datetime(match_date)
  corr_with_fantasy_points = all_player_match_wise_data[
      ['F_rm_3', 'F_ewm_3', 'F_rm_5', 'F_ewm_5', 'F_rm_10', 'F_ewm_10',
      'F_rm_15', 'F_ewm_15', 'I_rm_3', 'I_ewm_3', 'I_rm_5', 'I_ewm_5',
      'I_rm_10', 'I_ewm_10', 'I_rm_15', 'I_ewm_15', 'G_rm_3', 'G_ewm_3',
      'G_rm_5', 'G_ewm_5', 'G_rm_10', 'G_ewm_10', 'G_rm_15', 'G_ewm_15',
      'O_rm_3', 'O_ewm_3', 'O_rm_5', 'O_ewm_5', 'O_rm_10', 'O_ewm_10',
      'O_rm_15', 'O_ewm_15', 'Fantasy Points']
  ].corr()["Fantasy Points"].sort_values(ascending=False)
  high_correlation_features = corr_with_fantasy_points[corr_with_fantasy_points >= 0.05].index.tolist()
  filtered_data = all_player_match_wise_data[high_correlation_features]
  filtered_data['Opposition'] = all_player_match_wise_data['Opposition']
  filtered_data['Ground'] = all_player_match_wise_data['Ground']
  filtered_data['Date'] = all_player_match_wise_data['Date']
  filtered_data['Player ID'] = all_player_match_wise_data['Player ID']
  categorical_columns = ['Opposition', 'Ground', 'Date', 'Player ID']
  label_encoders = {}
  for col in categorical_columns:
      le = ExtendableLabelEncoder()
      filtered_data[col] = le.fit_transform(filtered_data[col])
      label_encoders[col] = le
  filtered_data
  X = filtered_data.drop(columns=["Fantasy Points"])
  y = filtered_data["Fantasy Points"]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)
  X_train_linear = X_train.drop(columns=categorical_columns)
  X_test_linear = X_test.drop(columns=categorical_columns)
  scaler_linear = StandardScaler()
  X_train_linear_scaled = scaler_linear.fit_transform(X_train_linear)
  X_test_linear_scaled = scaler_linear.transform(X_test_linear)
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)
  models = {
        "Linear Regression": LinearRegression(),
        "Random Forest": RandomForestRegressor(
            n_estimators=300,
            #min_samples_split=5,
            #max_depth=10,
            random_state=42
        ),
        "Gradient Boosting": GradientBoostingRegressor(
            n_estimators=100, max_depth=3, learning_rate=0.01, random_state=42
        ),
        "XGBoost": XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=1,
    tree_method='hist',
    deterministic=True,
    verbosity=0
),
        "Support Vector Regressor": SVR(
            C=1, epsilon=0.1, gamma='scale'
        )
    }


  results = {}
  predictions = {}


  for name, model in models.items():
      if name in ["Linear Regression", "Support Vector Regressor"]:
          model.fit(X_train_linear_scaled, y_train)
          y_pred = model.predict(X_test_linear_scaled)
      else:
          model.fit(X_train, y_train)
          y_pred = model.predict(X_test)

      predictions[name] = y_pred

      mae = mean_absolute_error(y_test, y_pred)
      mse = mean_squared_error(y_test, y_pred)
      r2 = r2_score(y_test, y_pred)

      results[name] = {"MAE": mae, "MSE": mse, "R² Score": r2}

  results_df = pd.DataFrame(results).T
  #print('Training 1 done')
  squad_data = csv_name
  final_news = pd.read_csv('final_new.csv')
  impacts_players_list = impacts(squad_data,date_of_match,match_no)
  impacts_players_list=[]
  x_factors = squad_data[squad_data['IsPlaying'] == 'X_FACTOR_SUBSTITUTE']
  probables = pd.read_csv('final_players.csv')
  probables = np.array(probables['Names'])
  for i in range(len(x_factors)):
    player_name = x_factors.iloc[i]['Player Name']
    if player_name in probables:
      impacts_players_list.append(player_name)
  #print(impacts_players_list)
  playing_players = squad_data[squad_data['IsPlaying'] == 'PLAYING']
  impact_players = squad_data[squad_data['Player Name'].isin(impacts_players_list)]
  squad_data = pd.concat([playing_players, impact_players]).drop_duplicates(subset=['Player Name']).reset_index(drop=True)
  name_to_id = final_news.set_index('Player Name')['Player ID'].to_dict()
  squad_data['Player ID'] = squad_data['Player Name'].map(name_to_id).astype('Int64')
  squad_ids = squad_data["Player ID"]
  team = squad_data['Team']
  squad_data['Opposition'] = squad_data['Team'].apply(lambda x: team.unique()[1] if x == team.unique()[0] else team.unique()[0])
  all_data_list = []

  for i in range((len(squad_data))):
      try:
          player_id = (squad_ids.iloc[i])
          player_data = extract_player_data(player_id)
          if len(player_data)<7:
            continue
          player_data = Batting_fantasy_points(player_data)
          player_data = Bowling_fantasy_points(player_data)
          player_data = Fielding_fantasy_points(player_data)
          player_data["Fantasy Points"] = (
              player_data["Batting Fantasy Points"]
              + player_data["Bowling Fantasy Points"]
              + player_data["Fielding Fantasy Points"]
              + 4
          )
          player_data['Date'] = pd.to_datetime(player_data['Date'], format="%d %b %Y")
          player_data['Wickets'] = player_data['Wkts'].fillna(0)
          inning=1
          if squad_data.iloc[i]['Team']!=batting_team:
            inning=2
          feature_values =  calculate_inning_feature(
                  player_id,
                  innings_counts,
                  inning,  # Convert to integer safely
                  player_data,
                  match_date,  # Convert to datetime safely
                  ground,
                   squad_data['Opposition'].iloc[i]
              )
          feature_values.update({
                  "Player ID": player_id,
                  "Date": match_date,
                  "Ground": ground,
                  "Opposition": squad_data['Opposition'].iloc[i],
                  "inning": inning,  # Ensure integer
                  # "Fantasy Points": player_data["Fantasy Points"].iloc[j]
              })

          all_data_list.append(feature_values)

      except :
          #print(f"Error processing Player ID {squad_ids.iloc[i]}:")
          c=4

  rcbvsgt = pd.DataFrame(all_data_list)
  date_of_match = pd.to_datetime(match_date)
  rcbvsgt = rcbvsgt[pd.to_datetime(rcbvsgt['Date'])==date_of_match]
  rcbvsgt['Original Player ID'] = rcbvsgt['Player ID']
  categorical_cols = ['Opposition', 'Ground', 'Date', 'Player ID']
  for col in categorical_cols:
      rcbvsgt[col] = label_encoders[col].transform(rcbvsgt[col])
  rcbvsgt_saved = rcbvsgt
  rcbvsgt_saved.fillna(0, inplace=True)
  rcbvsgt_saved.isna().sum()
  #rcbvsgt_X = rcbvsgt[high_correlation_features]
  rcbvsgt_X = rcbvsgt[
    [col for col in high_correlation_features if col != 'Fantasy Points']
  ]
  rcbvsgt_X_linear=rcbvsgt_X
  rcbvsgt_X['Opposition'] = rcbvsgt_saved['Opposition']
  rcbvsgt_X['Ground'] = rcbvsgt_saved['Ground']
  rcbvsgt_X['Date'] = rcbvsgt_saved['Date']
  rcbvsgt_X['Player ID'] = rcbvsgt_saved['Player ID']
  #rcbvsgt_X = rcbvsgt_X.drop(columns=['Fantasy Points'])
  rcbvsgt_X_linear = rcbvsgt_X_linear.drop(columns=categorical_cols)
  #rcbvsgt_y = rcbvsgt['Fantasy Points']
  scaler = StandardScaler()
  rcbvsgt_X_scaled = scaler.fit_transform(rcbvsgt_X_linear)
  rcbvsgt_X_scaled = scaler.transform(rcbvsgt_X_linear)
  #display(rcbvsgt_X_linear.columns,high_correlation_features)
  for name,model in models.items():
      if name in ["Support Vector Regressor",'Linear Regression']:
          y_pred = model.predict(rcbvsgt_X_scaled)
      else:
          y_pred = model.predict(rcbvsgt_X)

      rcbvsgt[name] = y_pred
  dream11_scores_for_match = {}
  player_id_to_name = squad_data.set_index("Player ID")["Player Name"].to_dict()
  pred_input = rcbvsgt
  id_to_type = squad_data.set_index("Player ID")["Player Type"].to_dict()
  pred_input["Type"] = pred_input["Original Player ID"].map(id_to_type)
  rcbvsgt['Type'] = rcbvsgt['Original Player ID'].map(id_to_type)
  return rcbvsgt,impacts_players_list

def constraints(merged_data, model):
    teams = merged_data['Opposition'].unique()
    team1_name, team2_name = teams[0], teams[1]

    wk_players = merged_data[merged_data["Type"] == "WK"]
    bowlers = merged_data[merged_data["Type"] == "BOWL"]
    batsmen = merged_data[merged_data["Type"] == "BAT"]
    all_rounders = merged_data[merged_data["Type"] == "ALL"]

    min_wk = wk_players.head(1)
    min_bowlers = bowlers.head(3)
    min_batsmen = batsmen.head(3)
    min_ar = all_rounders.head(1)

    best_min = pd.concat([min_wk, min_bowlers, min_batsmen, min_ar]).drop_duplicates(subset="Player ID")

    for team_name in [team1_name, team2_name]:
        while (best_min['Opposition'] == team_name).sum() < 4:
            pool = merged_data[(merged_data['Opposition'] == team_name) & ~merged_data['Player ID'].isin(best_min['Player ID'])]
            if pool.empty: break
            best_min = pd.concat([best_min, pool.head(1)])

    if len(best_min) < 11:
        rest = merged_data[~merged_data['Player ID'].isin(best_min['Player ID'])].sort_values(by=model, ascending=False)
        best_min = pd.concat([best_min, rest.head(11 - len(best_min))])
    elif len(best_min) > 11:
        best_min = best_min.sort_values(by=model, ascending=False).head(11)

    return best_min.sort_values(by=model, ascending=False).reset_index(drop=True)

def get_the_team(df):
  columns_to_avg = [
      'F_rm_3', 'F_ewm_3', 'F_rm_5', 'F_ewm_5', 'F_rm_10', 'F_ewm_10', 'F_rm_15', 'F_ewm_15',
      'I_rm_3', 'I_ewm_3', 'I_rm_5', 'I_ewm_5', 'I_rm_10', 'I_ewm_10', 'I_rm_15', 'I_ewm_15',
      'G_rm_3', 'G_ewm_3', 'G_rm_5', 'G_ewm_5', 'G_rm_10', 'G_ewm_10', 'G_rm_15', 'G_ewm_15',
      'O_rm_3', 'O_ewm_3', 'O_rm_5', 'O_ewm_5', 'O_rm_10', 'O_ewm_10', 'O_rm_15', 'O_ewm_15'
  ]
  cols = [
      #'Linear Regression'	,
      'Random Forest'	,
      'Gradient Boosting'	,
      'XGBoost'	,
      'Support Vector Regressor',
      'I_rm_15',
      'O_rm_5',
      'G_rm_3',
      'F_rm_15',
  ]

  sum1=0
  mapping = pd.read_csv('final_new.csv')
  id_name = mapping.set_index('Player ID')['Player Name'].to_dict()
  excluded_players = [1168049,822553,446763,793463,719719,290630,1175485,1131754,1159711,1350768,1292502,1350762,1148776,1339698,777515,1070196,1252585,1206052,1138316,974175,1123718,1161489,481896,1159843,1151286]
  def clean_avg1(row):
      values = row[columns_to_avg].values
      filtered = [v for v in values if v != 0]      # Remove 0s
      unique_vals = list(set(values))             # Remove duplicates
      return sum(unique_vals) / len(unique_vals) if unique_vals else 0  # Avoid division by 0

  def clean_avg(row):
      values = row[cols].values
      #filtered = [v for v in values if v != 0]      # Remove 0s
      unique_vals = list(set(values))             # Remove duplicates
      return sum(unique_vals) / len(unique_vals) if unique_vals else 0  # Avoid division by 0

  df['Avg'] = df.apply(clean_avg, axis=1)
  df = df[~df['Original Player ID'].isin(excluded_players)]
  top11_model = df.sort_values(by="Avg", ascending=False).reset_index(drop=True)
  best_11 = constraints(top11_model,'Avg')
  teams = best_11['Opposition'].unique()
  team1_name, team2_name = teams[0], teams[1]
  best_11['Name'] = best_11['Original Player ID'].apply(lambda x: id_name.get(x, 'Unknown'))
  def get_player_team(row):
    return team2_name if row['Opposition'] == team1_name else team1_name
  best_11['Team'] = best_11.apply(get_player_team, axis=1)
  dream_11_lohit = best_11[['Name', 'Team']]
  #(dream_11_lohit)
  return dream_11_lohit


def get_match_url(match_number, excel_path='Remaining matches.xlsx'):
    df = pd.read_excel(excel_path,engine='openpyxl')
    df['match_no'] = df['match_no'].astype(int)
    match_row = df[df['match_no'] == match_number]
    if match_row.empty:
        raise ValueError(f"Match number {match_number} not found in Excel file.")

    match_id = str(int(match_row.iloc[0]['id']))
    teams_part = match_row.iloc[0]['teams']

    # Construct the URL
    return f"https://www.cricbuzz.com/live-cricket-scorecard/{match_id}/tbd-vs-tbd-match-indian-premier-league-2025"

def print_match_details(match_number,teams):
    details = {}
    if match_number<=70:
      url = get_match_url(match_number, 'Remaining matches.xlsx')
    #print(f"Fetching from URL: {url}")
    elif match_number == 71:
      url = "https://www.cricbuzz.com/live-cricket-scores/118907/tbc-vs-tbc-qualifier-1-indian-premier-league-2025"
    elif match_number == 72:
      url = "https://www.cricbuzz.com/live-cricket-scores/118916/tbc-vs-tbc-eliminator-indian-premier-league-2025"
    elif match_number == 73:
      url = "https://www.cricbuzz.com/live-cricket-scores/118919/tbc-vs-tbc-qualifier-2-indian-premier-league-2025"
    elif match_number == 74:
      url = "https://www.cricbuzz.com/live-cricket-scores/118928/tbc-vs-tbc-final-indian-premier-league-2025"
    elif match_number == 75:
      url = "https://www.cricbuzz.com/live-cricket-scorecard/91740/kkr-vs-srh-final-indian-premier-league-2024"



    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Stadium
    stadium_tag = soup.find('span', itemprop='name')
    stadium_name = stadium_tag.get_text(strip=True).rstrip(',') if stadium_tag else "Unknown"
    #print("Stadium:", stadium_name)
    details['Stadium'] = stadium_name

    # Date
    date_tag = soup.find('span', itemprop='endDate')
    match_date = date_tag['content'] if date_tag else "Unknown"
    #print("Date:", match_date)
    details['Date'] = match_date
    # Match name (e.g. MI vs SRH)
    # match_info_div = soup.find('div', class_='cb-col cb-col-73')
    match_name = f'{teams[0]} vs {teams[1]}'
    #print("Match:", match_name)
    details['Match'] = match_name
    # Toss result
    info_items = soup.find_all('div', class_='cb-col cb-col-100 cb-mtch-info-itm')
    toss_result = "Unknown"
    for item in info_items:
        label = item.find('div', class_='cb-col cb-col-27')
        if label and label.get_text(strip=True) == 'Toss':
            toss_result = item.find('div', class_='cb-col cb-col-73').get_text(strip=True)
    #print("Toss:", toss_result)
    details['Toss'] = toss_result
    def extract_first_batting_team(toss_statement, match_name):
        team_name_map = {
            "KKR": "Kolkata Knight Riders",
            "CSK": "Chennai Super Kings",
            "MI": "Mumbai Indians",
            "LSG": "Lucknow Super Giants",
            "DC": "Delhi Capitals",
            "PBKS": "Punjab Kings",
            "RR": "Rajasthan Royals",
            "SRH": "Sunrisers Hyderabad",
            "GT": "Gujarat Titans",
            "RCB": "Royal Challengers Bengaluru"
        }

        reverse_map = {v: k for k, v in team_name_map.items()}

        toss_team = toss_statement.split(" won the toss")[0].strip()

        match_teams = match_name.replace(" ", "").split("vs")
        team1_full = team_name_map.get(match_teams[0], match_teams[0])
        team2_full = team_name_map.get(match_teams[1], match_teams[1])

        if "opt to bat" in toss_statement.lower() or "elected to bat" in toss_statement.lower():
            return toss_team
        elif "opt to bowl" in toss_statement.lower():
            return team2_full if toss_team == team1_full else team1_full
        else:
            return "Unknown"
    details['1st']=(extract_first_batting_team(toss_result,match_name))
    return details

stadium_mapping = {
    'Eden Gardens': 'Eden Gardens',
    'Rajiv Gandhi International Stadium': 'Hyderabad',
    'MA Chidambaram Stadium': 'Chennai',
    'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 'Visakhapatnam',
    'Narendra Modi Stadium': 'Ahmedabad',
    'Barsapara Cricket Stadium': 'Guwahati',
    'Wankhede Stadium': 'Wankhede',
    'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium': 'Lucknow',
    'M.Chinnaswamy Stadium': 'Bengaluru',
    'Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur': 'Mullanpur',
    'Sawai Mansingh Stadium': 'Jaipur',
    'Arun Jaitley Stadium': 'Delhi',
    'Himachal Pradesh Cricket Association Stadium' : 'Dharamsala',
}
team_mapping = {
    'Kolkata Knight Riders': 'KKR',
    'Royal Challengers Bengaluru': 'RCB',
    'Sunrisers Hyderabad': 'SRH',
    'Rajasthan Royals': 'RR',
    'Chennai Super Kings': 'CSK',
    'Mumbai Indians': 'MI',
    'Delhi Capitals': 'DC',
    'Gujarat Titans': 'GT',
    'Lucknow Super Giants': 'LSG',
    'Punjab Kings': 'PBKS',
    'CHE' : 'CSK',
    'Unknown': 'Unknown'
}


random.seed(42)
np.random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'
from datetime import datetime, timedelta



match_no = 74
match_no = int(match_no)
file_path = '/app/data/SquadPlayerNames_IndianT20League.xlsx'
sheet_name = f"Match_{match_no}"
#csv = pd.read_excel(file_path, sheet_name=sheet_name,engine='openpyxl')
csv = pd.read_csv(f'SquadPlayerNames_IndianT20League - Match_{match_no}.csv')
teams = csv['Team'].unique()
match_no-=2
details = print_match_details(match_no,teams)
print(details['Date'])
#details['Date']= '2025-05-06'
Date = datetime.strptime(details['Date'], '%Y-%m-%d')
Ground = stadium_mapping[details['Stadium']]
#print(details)
batting_first = team_mapping[details['1st']]
if batting_first == 'Unknown':
  batting_first = csv['Team'].unique()[0]
  if batting_first == 'CHE':
    batting_first = 'CSK'
batting_first = team_mapping[details['1st']]
data_till_match = training_pipeline(Date)
data_till_match['Date'] = pd.to_datetime(data_till_match['Date']).dt.strftime('%Y-%m-%d')
data_till_match['inning'] = data_till_match['inning'].astype(int)
squad_data,impact_players = pipeline_squad_data(Date,csv,data_till_match,batting_first,Ground,match_no)
dream_11 = get_the_team(squad_data)
ids = dream_11['Name']
team_list = [csv[csv['Player Name'] == x]['Team'].values[0] for x in ids]
player_names = [csv[csv['Player Name'] == x]['Player Name'].values[0] for x in ids]
dream_11['Team'] = team_list
dream_11['Player name'] = player_names
dream_11['C/VC'] = ['C', 'VC'] + ['NA'] * (len(dream_11) - 2)
dream_11_info = dream_11[['Player name','Team','C/VC']]
impact_indices = []
for i in range(2):
    if dream_11_info.iloc[i]['Player name'] in impact_players:
        impact_indices.append(i)

# Find the indices of the top 2 players who are impact players
top_two_players = dream_11_info.iloc[:2]['Player name'].tolist()
impact_indices = [i for i, name in enumerate(top_two_players) if name in impact_players]

# Get eligible (non-impact) players from lower in the list
eligible_players = dream_11_info[~dream_11_info['Player name'].isin(impact_players + top_two_players)]

replacement_idx = 0

for i in impact_indices:
    if replacement_idx < len(eligible_players):
        # Get replacement player and their index
        replacement_player = eligible_players.iloc[replacement_idx]
        replacement_player_index = replacement_player.name  # original index in dream_11_info

        # Swap rows i (impact in top 2) and replacement_player_index (non-impact below)
        temp = dream_11_info.iloc[i].copy()
        dream_11_info.iloc[i] = dream_11_info.loc[replacement_player_index]
        dream_11_info.loc[replacement_player_index] = temp
        # Update C/VC flags
        dream_11_info.at[i, 'C/VC'] = 'C' if i == 0 else 'VC'
        dream_11_info.at[replacement_player_index, 'C/VC'] = 'NA'
        replacement_idx += 1
print(dream_11_info)

# output_filename = f"c_block_warriors_output.csv"
# print(f"Output csv name: {output_filename}")
# print(dream_11_info.to_string(index=False))
# output_path = f"/app/data/c_block_warriors_output.csv"
# dream_11_info.to_csv(output_path, index=False)
# print(f"Saved output CSV to {output_path}")



In [ ]:
squad_data.sort_values(by='F_ewm_3', ascending=False,inplace=True)
playermapps = pd.read_csv('final_new.csv')
playermapps = playermapps.set_index('Player ID')['Player Name'].to_dict()
print(playermapps)
for index,row in squad_data.iterrows():
  print(playermapps[int(row['Original Player ID'])])